In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from geopandas import GeoDataFrame
from sklearn import preprocessing

plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.labelsize'] = 12

### El archivo usado es el de los 6 meses de 2017

In [2]:
propiedades = pd.read_csv('properati.csv')

In [3]:
propiedades.loc[:, 'mes'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[1]))

In [4]:
propiedades.drop(['operation', 'country_name', 'geonames_id', 'lat-lon', 'properati_url', 'image_thumbnail'],axis =1,inplace =True)

In [5]:
propiedades = propiedades.loc[propiedades.state_name.str.contains('Capital | G.B.A'),:]

In [6]:
def unificar_superficie(fila):
    s1,s2 = fila
    if (s2 and not np.isnan(s2)):
        return s2
    if (s1 and not np.isnan(s1)):
        return s1
    return np.nan

In [7]:
propiedades.loc[:,'superficie'] = propiedades.loc[:,['surface_covered_in_m2', 'surface_total_in_m2']].apply(unificar_superficie, axis = 1)
propiedades.drop(['surface_covered_in_m2', 'surface_total_in_m2'], axis = 1, inplace = True)

In [8]:
def calcularPrecioM2(fila):
    """Recibe una lista con precio total, superficie y precio por metro cuadrado,
    y calcula este ultimo a partir de los anteriores, cuando es posible"""
    precio_total, precio_m2, superficie = fila
    if (np.isnan(precio_m2)):
        if (superficie and not np.isnan(superficie)):
            return precio_total/superficie
    return precio_m2

In [9]:
propiedades.loc[:,'price_usd_per_m2'] = propiedades.loc[:,['price_aprox_usd','price_usd_per_m2', 'superficie']].\
            apply(calcularPrecioM2,axis = 1)

In [10]:
def obtener_barrio(fila):
    barrio,padre = fila
    return padre.split('|')[3]

In [11]:
propiedades.loc[:, 'place_name'] = propiedades.loc[:, ['place_name','place_with_parent_names']].apply(lambda x: obtener_barrio(x), axis = 1)

In [12]:
subtes = gpd.read_file('subtes.shp')

In [13]:
propiedades.loc[:,'ditancia_subtes'] = propiedades.loc[:, ['lon', 'lat']].apply(lambda x: (min(subtes.distance(Point(x[0], x[1]))))*100, axis = 1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [14]:
def tiene_cadena(cadena, lista):
    cadena = str(cadena)
    cadena = cadena.lower()
    resultado = False
    for x in lista:
        resultado = resultado or (x in cadena)
    return resultado

In [15]:
cadenas = ['vigilancia', 'seguridad', 'alarma', 'blindado', 'blindaje', 'guardia']
propiedades.loc[:,'seguridad'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [16]:
cadenas = ['gimnasio', 'gym', 'gimnacio']
propiedades.loc[:,'gimnasio'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [17]:
propiedades.to_csv('properati.csv', index = False)

## Codifico el tipo de propiedad

In [18]:
tipo_propiedad = {'PH': 0, 'apartment':587, 'house': 1829, 'store': 28531}

In [19]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipo_propiedad[x]) 

## Codifico el barrio

In [20]:
import random
barrios = {}
x = 0
for i in propiedades.groupby('place_name').agg(np.mean).lat.keys():
    if i in barrios:
        continue
    barrios[i] = x
    x+=random.randint(1, 100)

In [23]:
propiedades.reset_index(drop = True, inplace = True)

In [24]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: barrios[x]) 

In [25]:
adivinar = propiedades.loc[propiedades.mes >= 7,:]

In [26]:
entrenar = propiedades.loc[propiedades.mes < 7,:]

In [27]:
entrenar.keys()

Index([u'id', u'created_on', u'property_type', u'place_name',
       u'place_with_parent_names', u'state_name', u'lat', u'lon', u'price',
       u'currency', u'price_aprox_local_currency', u'price_aprox_usd',
       u'price_usd_per_m2', u'price_per_m2', u'floor', u'rooms', u'expenses',
       u'description', u'title', u'mes', u'superficie', u'ditancia_subtes',
       u'seguridad', u'gimnasio'],
      dtype='object')

In [44]:
condiciones = []
precios = []

for i in range(0,len(entrenar)):
    if np.isnan(entrenar.price_aprox_usd[i]):
        continue
    if np.isinf(entrenar.ditancia_subtes[i]):
        continue
    l = []
    l.append(entrenar.place_name[i])
    l.append(entrenar.property_type[i])
    l.append(entrenar.ditancia_subtes[i])
    l.append(entrenar.seguridad[i])
    l.append(entrenar.gimnasio[i])
    condiciones.append(l)
    precios.append(entrenar.price_aprox_usd[i])

In [29]:
def adivinar_precio(fila, estimador):
    barrio,tipo,subte, seg, gim = fila
    if np.isinf(subte):
        return 0 # Ver de resolver estos despues
    return  estimador.predict([[barrio,tipo,subte,seg,gim]])[0]

## Con SVM

In [30]:
from sklearn import svm

In [50]:
clf = svm.SVR(degree = 2)
clf.fit(condiciones, precios)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [46]:
entrenar[['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio', 'price_aprox_usd']].head()

,place_name,property_type,ditancia_subtes,seguridad,gimnasio,price_aprox_usd
0,2234,0,5.004808,False,False,62000.0
1,1926,587,50.029804,False,False,150000.0
2,2234,587,5.753307,False,False,72000.0
3,2064,0,4.980840,False,False,95000.0
4,4184,0,5.444071,False,False,130000.0


In [51]:
clf.predict([[2234,0,5.004808,False,False]])

array([ 155001.95519083])

In [52]:
adivinar.loc[:,'precio_adivinado'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x,clf), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [54]:
adivinar.loc[:,'error_porcentual_svm'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [57]:
adivinar.loc[adivinar.precio_adivinado != 0].error_porcentual_svm.describe()

count    16478.000000
mean        59.062793
std         96.719020
min          0.000062
25%         22.480685
50%         46.437901
75%         74.035552
max       2970.150732
Name: error_porcentual_svm, dtype: float64

In [69]:
adivinar.loc[adivinar.error_porcentual_svm > 60000].head()[['precio_adivinado','price_usd_per_m2', 'price_aprox_usd', 'superficie']]

#el precio que tiene es un asco pero es solo este

,precio_adivinado,price_usd_per_m2,price_aprox_usd,superficie
52474,2153.467798,3.208556,600000.0,187000.0


## Con un árbol

In [58]:
from sklearn.tree import DecisionTreeRegressor

In [59]:
# Primero uso con profundidad 2
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_1.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [60]:
adivinar.loc[:,'precio_adivinado_arbol_2'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, regr_1), axis = 1)

In [81]:
adivinar.loc[:,'error_porcentual_arbol_2'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_2']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [82]:
adivinar.error_porcentual_arbol_2.describe()

count    39967.000000
mean        97.988472
std        102.612493
min          0.091854
25%         79.301381
50%        100.000000
75%        100.000000
max       5046.181323
Name: error_porcentual_arbol_2, dtype: float64

#### El error es mucho mayor SVM

In [63]:
# Pruebo con profundidad 4
regr_2 = DecisionTreeRegressor(max_depth=4)
regr_2.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [64]:
adivinar.loc[:,'precio_adivinado_arbol_4'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, regr_2), axis = 1)

In [84]:
adivinar.loc[:,'error_porcentual_arbol_4'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_4']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [85]:
adivinar.error_porcentual_arbol_4.describe()

count    39967.000000
mean        96.459066
std        123.610419
min          0.003735
25%         72.318061
50%        100.000000
75%        100.000000
max       7420.366078
Name: error_porcentual_arbol_4, dtype: float64

#### Disminuyo pero muy poco

In [67]:
# Pruebo con profundidad 10
regr_3 = DecisionTreeRegressor(max_depth=10)
regr_3.fit(condiciones, precios)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [68]:
adivinar.loc[:,'precio_adivinado_arbol_10'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, regr_3), axis = 1)

In [86]:
adivinar.loc[:,'error_porcentual_arbol_10'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_10']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [87]:
adivinar.error_porcentual_arbol_10.describe()

count    39967.000000
mean        89.748897
std        128.997313
min          0.000000
25%         52.418749
50%        100.000000
75%        100.000000
max       8166.666667
Name: error_porcentual_arbol_10, dtype: float64

In [125]:
# Pruebo con varias profundidades
# Tarda un poco
l =[]
for i in range (1, 100):
    regr_4 = DecisionTreeRegressor(max_depth=i)
    regr_4.fit(condiciones, precios)

    adivinar.loc[:,'precio_adivinado_arbol_1000'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
    .apply(lambda x: adivinar_precio(x, regr_4), axis = 1)

    adivinar.loc[:,'error_porcentual_arbol_1000'] = adivinar.loc[:, ['price_usd_per_m2', 'precio_adivinado_arbol_1000']]\
    .apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

    l.append([adivinar.error_porcentual_arbol_1000.mean(),i])


In [126]:
sorted(l)[0]

[97.48518322662845, 15]

In [88]:
regr_4 = DecisionTreeRegressor(max_depth=15)
regr_4.fit(condiciones, precios)

adivinar.loc[:,'precio_adivinado_arbol_1000'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, regr_4), axis = 1)

adivinar.loc[:,'error_porcentual_arbol_1000'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_arbol_1000']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [89]:
adivinar.error_porcentual_arbol_1000.describe()

count    39967.000000
mean        91.004964
std        139.755834
min          0.000000
25%         50.657891
50%        100.000000
75%        100.000000
max       8166.666667
Name: error_porcentual_arbol_1000, dtype: float64

#### Por lo que vi no da siempre lo mismo

## Con Gradient Boosting regresor

In [91]:
from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.9,max_depth=4,\
                                random_state=0, loss='ls')

In [92]:
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [75]:
est.predict([[521, 587, 1.0223190036801129, False, False]])

array([ 297879.42665234])

In [93]:
adivinar.loc[:,'precio_adivinado_gradient'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, est), axis = 1)

In [94]:
adivinar.loc[:,'error_porcentual_gradient'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [95]:
adivinar.loc[adivinar.precio_adivinado_gradient>1000].error_porcentual_gradient.describe()

count    15879.000000
mean        98.802897
std        400.277998
min          0.009231
25%         20.928640
50%         46.307166
75%         83.428883
max      27246.505502
Name: error_porcentual_gradient, dtype: float64

In [96]:
#Pruebo con otros parametros
est = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.9,max_depth=4,\
                                random_state=0, loss='ls')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10000, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [97]:
adivinar.loc[:,'precio_adivinado_gradient2'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, est), axis = 1)

In [98]:
adivinar.loc[:,'error_porcentual_gradient2'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient2']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [99]:
adivinar.loc[adivinar.precio_adivinado_gradient>1000].error_porcentual_gradient2.describe()

count    15879.000000
mean       104.612159
std        406.213040
min          0.000029
25%         21.554218
50%         48.899332
75%         90.843587
max      27250.599332
Name: error_porcentual_gradient2, dtype: float64

In [100]:
#Pruebo con mayor profundidad y otra funcion loss
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.9,max_depth=10,\
                                random_state=0, loss='huber')
est.fit(condiciones, precios)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [101]:
adivinar.loc[:,'precio_adivinado_gradient3'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio(x, est), axis = 1)

In [102]:
adivinar.loc[:,'error_porcentual_gradient3'] = adivinar.loc[:, ['price_aprox_usd', 'precio_adivinado_gradient3']]\
.apply(lambda x: abs(x[0]-x[1])*100/(x[0]), axis = 1)

In [103]:
adivinar.loc[adivinar.precio_adivinado_gradient>1000].error_porcentual_gradient3.describe()

count    15879.000000
mean        95.716766
std        266.434269
min          0.000000
25%         19.958107
50%         45.624867
75%         87.411030
max      18940.656360
Name: error_porcentual_gradient3, dtype: float64

#### Probar lo mismo pero usando el precio ya que en algunos vamos a tener que hacer eso porque no tienen precio por metro cuadrado. Ver como hacer cuando faltan datos como latitud y longitud, entre otros

## Cosas para ver como usarlas

Las transformaciones de los tipos de propiedades y barrios tengo entendido que deberían ser bien realizados porque sino perjudican en la prediccion. Encontre algo que sirve para convertir los indices numericos que le establecimos en algo que "no afecta la prediccion". El problema es que no me deja meter esos valores obtenidos en los algoritmos de prediccion.
Habria que ver si hay una forma de meterlo. 
Se hace con esto:

In [259]:
# ejemplo usando los tipos de propiedades. Reemplazo los tipos por los indices 0, 1, 2 y 3
from sklearn import preprocessing
tipos = preprocessing.OneHotEncoder()
tipos.fit([[0], [1], [2], [3]]) 

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [268]:
# basicamente desplaza ese 1
print(tipos.transform([[0]]).toarray())
print(tipos.transform([[1]]).toarray())

# Me olvide de intentar meter lo que devuelve sin aplicarle el toarray(). Tal vez acepte esa otra estructura

[[ 1.  0.  0.  0.]]
[[ 0.  1.  0.  0.]]


Ver de usar nearest neighbors en conjunto con otros algoritmos

In [237]:
#from sklearn.neighbors import NearestNeighbors
#nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(condiciones)
#gnb.fit(condiciones, precios)

In [160]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(['house', 'apartment', 'ph', 'store'])

LabelEncoder()

In [165]:
encoder.transform(['store'])

array([3])